In [1]:
import os
import random

import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import matplotlib.pyplot as plt
import pandas as pd

## Get Data Streams

In [2]:
DATA_ROOT = "/home/mauricio/data"

file_list = os.listdir(DATA_ROOT)
file_list.sort()
file_groups = {}

for file in file_list:
    file_group = file.split("_")[0]
    if file_group not in file_groups:
        file_groups[file_group] = []
    file_groups[file_group].append(file)

sensor_data = {}

for file_group in file_groups:
    files = file_groups[file_group]
    for file in files:
        file_data = pd.read_csv(os.path.join(DATA_ROOT, file), parse_dates=["timestamp"])
        #file_data["x_tick"] = file_data["timestamp"].apply(lambda x: x.strftime("%H:%M"))
        file_data = file_data.set_index("timestamp")
        if file_group not in sensor_data:
            sensor_data[file_group] = file_data
        else:
            sensor_data[file_group] = pd.concat([sensor_data[file_group], file_data])   

## Generate Sample Position Data

In [3]:
initial = 0.0
sample = []
for i in range(sensor_data["positions"]["lat"].shape[0]):
    sample.append(initial)
    initial = initial + random.randrange(-1.0, 1.0)
sensor_data["positions"]["lat"] = sensor_data["positions"]["lat"] + sample
initial = 0.0
sample = []
for i in range(sensor_data["positions"]["lat"].shape[0]):
    sample.append(initial)
    initial = initial + random.randrange(-1.0, 1.0)
sensor_data["positions"]["lon"] = sensor_data["positions"]["lon"] + sample

## Plot Results

In [20]:
def display_sensor_data(start, end):
    for group in sensor_data:
        data_subset = sensor_data[group]
        data_subset = data_subset[(data_subset.index >= start) & (data_subset.index <= end)]
        if group == "positions":
            plt.plot(data_subset["lon"], data_subset["lat"])
        else:
            plt.plot(data_subset[data_subset.columns])
        plt.title(group)
        plt.xticks(rotation=45)
        plt.show()


@interact
def slice_sensor_data(start=sensor_data["pressure"].index, end=sensor_data["pressure"].index):
    display_sensor_data(start, end)



interactive(children=(Dropdown(description='start', options=(Timestamp('2022-07-02 20:51:08.496993'), Timestam…